In [ ]:
import pyodbc
import json

# Cargar configuración desde el archivo JSON
with open('keys.json') as config_file:
    config = json.load(config_file)

# Set up your connection string
server = config['server']
database = config['database']
username = config['username']
password = config['password']

# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")

Connected to the Azure SQL Database successfully!


In [46]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Execute query and save data into a DataFrame
import pandas as pd
query = "SELECT * FROM SalesLT.Customer"

df = pd.read_sql(query, conn)

# Library for regression ML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Drop 'Title' , 'FirstName', 'LastName', 'EmailAddress', 'Phone', 'PasswordHash', 'PasswordSalt', 'NameStyle', 'MiddleName', 'Suffix' columns
df.drop(columns=['CustomerID', 'Title', 'FirstName', 'LastName', 'EmailAddress', 'Phone', 'PasswordHash', 'PasswordSalt', 'NameStyle', 'MiddleName', 'Suffix', 'rowguid', 'CompanyName', 'SalesPerson'], inplace=True)

# COnvert 'ModifiedDate' to datetime and separate into year, month, day
df['ModifiedDate'] = pd.to_datetime(df['ModifiedDate'])
df['ModifiedDate_year'] = df['ModifiedDate'].dt.year
df['ModifiedDate_month'] = df['ModifiedDate'].dt.month
df['ModifiedDate_day'] = df['ModifiedDate'].dt.day
df.drop(columns=['ModifiedDate'], inplace=True)

# Give me first day and last day of this dataset
print("First day of the dataset: ", df['ModifiedDate_year'].min(), df['ModifiedDate_month'].min(), df['ModifiedDate_day'].min())
print("Last day of the dataset: ", df['ModifiedDate_year'].max(), df['ModifiedDate_month'].max(), df['ModifiedDate_day'].max())

print(df.head(20))

First day of the dataset:  2005 1 1
Last day of the dataset:  2009 12 16
    ModifiedDate_year  ModifiedDate_month  ModifiedDate_day
0                2005                   8                 1
1                2006                   8                 1
2                2005                   9                 1
3                2006                   7                 1
4                2006                   9                 1
5                2007                   9                 1
6                2006                   7                 1
7                2006                   9                 1
8                2005                   8                 1
9                2006                   8                 1
10               2006                   9                 1
11               2005                   8                 1
12               2007                   4                 1
13               2005                   9                 1
14               2006      

In [48]:
import pandas as pd

# Paso 1: Generar todas las fechas del rango completo
all_dates = pd.date_range(start='2005-01-01', end='2009-12-16', freq='D')
df_all = pd.DataFrame({
    'ModifiedDate_year': all_dates.year,
    'ModifiedDate_month': all_dates.month,
    'ModifiedDate_day': all_dates.day
})

# Paso 2: Asegurar que las fechas originales están bien formateadas
df['label'] = 1  # Marcar las fechas existentes como 1

# Paso 3: Hacer el merge (left join)
df_full = df_all.merge(df[['ModifiedDate_year', 'ModifiedDate_month', 'ModifiedDate_day', 'label']],
                       on=['ModifiedDate_year', 'ModifiedDate_month', 'ModifiedDate_day'],
                       how='left')

# Paso 4: Rellenar fechas faltantes con 0
df_full['label'] = df_full['label'].fillna(0).astype(int)

# Resultado: df_full tiene una fila por cada día entre 2005-01-01 y 2009-12-16
print(df_full.head())

   ModifiedDate_year  ModifiedDate_month  ModifiedDate_day  label
0               2005                   1                 1      0
1               2005                   1                 2      0
2               2005                   1                 3      0
3               2005                   1                 4      0
4               2005                   1                 5      0


In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Paso 1: Seleccionar características (features) y la variable objetivo (label)
X = df_full[['ModifiedDate_year', 'ModifiedDate_month', 'ModifiedDate_day']]
y = df_full['label']

# Paso 2: Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Paso 3: Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Paso 4: Evaluar el modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       357
           1       1.00      0.99      0.99       170

    accuracy                           1.00       527
   macro avg       1.00      0.99      1.00       527
weighted avg       1.00      1.00      1.00       527

